#### Install required libraries


In [1]:
! pip install zoish --force-reinstall
! pip install feature-engine gpboost category-encoders scikit-learn ipywidgets numpy pandas --force-reinstall

  Obtaining dependency information for zoish from https://files.pythonhosted.org/packages/25/33/957a528a4f040f539a00a0fcc1e4cc39046c9675b4d2c0bb30b57f98ea3c/zoish-4.6.0-py3-none-any.whl.metadata
  Using cached zoish-4.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached appnope-0.1.3-py2.py3-none-any.whl (4.4 kB)
  Obtaining dependency information for argcomplete==3.1.1 from https://files.pythonhosted.org/packages/4f/ef/8b604222ba5e5190e25851aa3a5b754f2002361dc62a258a8e9f13e866f4/argcomplete-3.1.1-py3-none-any.whl.metadata
  Using cached argcomplete-3.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl (11 kB)
  Obtaining dependency information for catboost==1.2.1 from https://files.pythonhosted.org/packages/c1/bb/ea618c1192f926dcf51228eabe073695e9acede9d00205d0bd14bbba0654/catboost-1.2.1-cp37-cp37m-macosx_11_0_universal2.whl.metadata
  Using cached catboost-1.2.1-cp37-cp37m-macosx_11_0_universal2.whl.metadata (1.2 kB)
  Obtaining dependency informa

In [ ]:
# Importing built-in libraries
import pandas as pd  # For data manipulation and analysis
import sys  # For accessing system-specific parameters and functions
import zoish  # Assuming it's a custom library for your project
import sklearn  # For machine learning models
import numpy  # For numerical computations
import gpboost

# Importing scikit-learn utilities for various ML tasks
from sklearn.compose import ColumnTransformer  # For applying transformers to columns
from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier
from sklearn.impute import SimpleImputer  # For handling missing data
from sklearn.metrics import (  # For evaluating the model
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
)
from sklearn.model_selection import GridSearchCV, train_test_split  # For CV and splitting dataset
from sklearn.pipeline import Pipeline  # For creating ML pipelines
from sklearn.preprocessing import StandardScaler  # For feature scaling

# Importing other third-party libraries
from category_encoders import TargetEncoder  # For encoding categorical variables
from zoish.feature_selectors.shap_selectors import (  # For feature selection and visualization
    ShapFeatureSelector,
    ShapPlotFeatures,
)
import logging  # For logging events and errors

# Configuring logging settings
from zoish import logger  # Assuming it's a custom logger from zoish
logger.setLevel(logging.ERROR)  # Set logging level to ERROR

# Importing feature imputation library
from feature_engine.imputation import MeanMedianImputer  # For imputing mean/median

# Re-setting logging level (this seems redundant, consider keeping only one)
logger.setLevel(logging.ERROR)

# Printing versions of key libraries for debugging and documentation
print(f'Python version : {sys.version}')
print(f'zoish version : {zoish.__version__}')
print(f'sklearn version : {sklearn.__version__}')
print(f'pandas version : {pd.__version__}')  # Using the alias for pandas
print(f'numpy version : {numpy.__version__}')
print(f'gpboost version : {gpboost.__version__}')


#### Example: Audiology (Standardized) Data Set
###### https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29


#### Read data


In [ ]:
import gpboost as gpb
import numpy as np
np.random.seed(42)
n = 100
num_features = 10
X = pd.DataFrame(np.random.rand(n, num_features), columns=[f'feature_{i}' for i in range(num_features)])
group = pd.Series(np.random.randint(0, 10, n), name='group')
y = pd.Series(1.0 + 2.0 * X.iloc[:, 0] + np.random.normal(0, 1, n), name='target')

# Create and train the GPBoost model
model = gpb.GPModel(group_data=group)
model.fit(y=y, X=X)

# Assuming `y` is your target variable
# Check if `y` is 1-dimensional
if len(y.shape) > 1:
    # Reshape `y` to be 1-dimensional
    y = y.ravel()

# Ensure `y` is a numpy array
y = np.array(y)
# Make predictions
#y_pred = model.predict(X)

# Output the predictions
#print("Predictions:", y_pred)


#### Defining the feature pipeline steps:
Here, we use an untuned XGBClassifier model with the ShapFeatureSelector.In the next section, we will repeat the same process but with a tuned XGBClassifier. The aim is to demonstrate that a better estimator can yield improved results when used with the ShapFeatureSelector.


In [ ]:
#estimator_for_feature_selector= XGBClassifier()     
#estimator_for_feature_selector.fit(X_train, y_train)
shap_feature_selector = ShapFeatureSelector(model=model, num_features=5, cv = 5, scoring='accuracy', direction='maximum', n_iter=10, algorithm='auto')
        
# Define pre-processing for numeric columns (float and integer types)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Define pre-processing for categorical features
categorical_features = X.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', TargetEncoder(handle_missing='return_nan'))])

# Combine preprocessing into one column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Feature Selection using ShapSelector 
feature_selection = shap_feature_selector 

# Classifier model
classifier = gpb.GPModel(group_data=group)

# Create a pipeline that combines the preprocessor with a feature selection and a classifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('feature_selection', feature_selection),
                           ('classifier', classifier)])

# Fit the model
pipeline.fit(X, y)

# Predict on test data
y_pred = pipeline.predict(X)

# Output first 10 predictions
print(y_pred[:5])

#### Check performance of the Pipeline


In [ ]:

print("F1 score : ")
print(f1_score(y_test, y_test_pred,average='micro'))
print("Classification report : ")
print(classification_report(y_test, y_test_pred))
print("Confusion matrix : ")
print(confusion_matrix(y_test, y_test_pred))




#### Use better estimator:
In this iteration, we will utilize the optimally tuned estimator with the ShapFeatureSelector, which is expected to yield improved results."

In [ ]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()


In [ ]:

# Define the XGBClassifier
xgb_clf = XGBClassifier()

# Define the parameter grid for XGBClassifier
param_grid = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [ 4, 5],
    'min_child_weight': [1, 2, 3],
    'gamma': [0, 0.1, 0.2],
}

# Define the scoring function
scoring = make_scorer(f1_score, average='micro')  # Use 'micro' average in case of multiclass target

# Set up GridSearchCV
grid_search = GridSearchCV(xgb_clf, param_grid, cv=5, scoring=scoring, verbose=1)
grid_search.fit(X_train, y_train)
# Fit the GridSearchCV object
estimator_for_feature_selector= grid_search.best_estimator_ 
shap_feature_selector = ShapFeatureSelector(model=estimator_for_feature_selector, num_features=5, scoring='accuracy', algorithm='auto',cv = 5, n_iter=10, direction='maximum')


pipeline =Pipeline([
            # int missing values imputers
            ('floatimputer', MeanMedianImputer(
                imputation_method='mean', variables=int_cols)),
           
            ('shap_feature_selector', shap_feature_selector),
            ('classfier', RandomForestClassifier(n_estimators=100))


 ])


# Fit the model
pipeline.fit(X_train, y_train)

# Predict on test data
y_test_pred = pipeline.predict(X_test)

# Output first 10 predictions
print(y_test_pred[:10])
            

#### Performance has improved

In [ ]:

print("F1 score : ")
print(f1_score(y_test, y_test_pred,average='micro'))
print("Classification report : ")
print(classification_report(y_test, y_test_pred))
print("Confusion matrix : ")
print(confusion_matrix(y_test, y_test_pred))



#### Shap related plots

In [ ]:

# Plot the feature importance
plot_factory = ShapPlotFeatures(shap_feature_selector) 
plot_factory.summary_plot()


In [ ]:
plot_factory.summary_plot_full()

In [ ]:

# Plot the feature importance
plot_factory.bar_plot()

In [ ]:
plot_factory.bar_plot_full()

In [ ]:
plot_factory.dependence_plot('special forms')